In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!git clone https://github.com/pingpong-ai/chatspace.git

Cloning into 'chatspace'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 196 (delta 4), reused 4 (delta 1), pack-reused 172
Receiving objects: 100% (196/196), 9.15 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [0]:
import torch
print(torch.__version__)

1.3.1


In [0]:
!pip install pyahocorasick

     |████████████████████████████████| 317kB 2.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81704 sha256=bc832e8d3358998a3eabad50477b14697abff4f0b2d9d27036f478648547eff6
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [0]:
from chatspace.chatspace import inference

spacer = inference.ChatSpace()
a = '3급장애인 연금 공단이라고 해서 이사 를 가도 지급 이 됩니다'
spacer.space(a.replace(" ", ""),custom_vocab=["급장애인"])
# spacer.space(a)
# '안녕 만나서 반가워 내 이름은 뽀로로라고 해'


Loading JIT Compiled ChatSpace Model


'3급장애인연금 공단이라고 해서 이사를 가도 지급이 됩니다'

## RAWS

In [3]:
!pip install fasttext==0.8.3
!pip install -upgrade keras==keras==2.1.2
!pip install -upgrade tensorflow-gpu==1.4.1



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [4]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)


Using TensorFlow backend.


1.15.0
2.2.5


AttributeError: ignored

In [6]:
import numpy as np
import sys
import fasttext
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.01
set_session(tf.Session(config=config))

from keras.models import Sequential, Model
from keras.layers import Input, Embedding, LSTM, GRU, SimpleRNN, Dense, Lambda
import keras.backend as K
from keras.callbacks import ModelCheckpoint
import keras.layers as layers

from keras import optimizers
adam_half = optimizers.Adam(lr=0.0005)

from keras.preprocessing import sequence
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.embeddings import Embedding

from random import random
from numpy import array
from numpy import cumsum
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
import re

dic_kor = fasttext.load_model('/content/drive/My Drive/model_kor.bin')
def loadvector(File):
    f = open(File,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    return model
# dic_eng = loadvector('vectors/model_eng.txt')

import string
idchar = {}
for i in range(len(string.ascii_lowercase)):
  idchar.update({string.ascii_lowercase[i]:i})

for i in range(10):
  idchar.update({i:i+26})

idchar.update({'#':36})

big  = re.compile(r"[A-Z]")
small= re.compile(r"[a-z]")
num  = re.compile(r"[0-9]")

from keras.models import load_model
model_kor = load_model('/content/drive/My Drive/model_kor.hdf5')
# model_eng = load_model('model/model_eng.hdf5')

## Functions_KOR

threshold_kor=0.5
overlap=30

def pred_correction_rnn(sent,model,dic,maxlen,wdim):
    conv = np.zeros((1,maxlen,wdim,1))
    rnn  = np.zeros((1,maxlen,wdim))
    charcount = -1
    for j in range(len(sent)):
      if j<maxlen and sent[j]!=' ':
        charcount=charcount+1
        conv[0][charcount,:,0]=dic[sent[j]]
        rnn[0][charcount,:]=dic[sent[j]]
    z = model.predict([conv,rnn])[0]
    sent_raw = ''
    count_char=-1
    lastpoint=-1
    lastchar=-1
    for j in range(len(sent)):
      if sent[j]!=' ':
        count_char=count_char+1
        sent_raw = sent_raw+sent[j]
        if z[count_char]>threshold_kor:
          sent_raw = sent_raw+' '
          if j<overlap:
            lastpoint=len(sent_raw)
            lastchar=j
    return sent_raw, lastpoint, lastchar

def kor_spacing(s):
    if len(s)<overlap:
      temp,lp,lc = pred_correction_rnn(s,model_kor,dic_kor,100,100)
      z = temp+"\n"
    else:
      z=''
      start=0
      while start<len(s):
        if start+overlap<len(s):
          temp,lp,lc =pred_correction_rnn(s[start:start+2*overlap],model_kor,dic_kor,100,100)
          temp=temp[:lp]
        else:
          temp,lp,lc =pred_correction_rnn(s[start:],model_kor,dic_kor,100,100)
          lc = overlap
        z = z+temp
        start=start+lc+1
      z = z+"\n"
    return z






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [7]:
kor_spacing("이제집에가고싶어요")

'이제 집에 가고 싶어요 \n'

In [0]:
file_dir = '/content/drive/My Drive/vectors.zip'

In [0]:
!pip install --upgrade fasttext==0.8.3

     |████████████████████████████████| 81kB 5.2MB/s 
  Created wheel for fasttext: filename=fasttext-0.8.3-cp36-cp36m-linux_x86_64.whl size=1092625 sha256=727706221e835b3d8474290635e185ccb7b76f6bf05a37e5bd483dcd16a67818
  Stored in directory: /root/.cache/pip/wheels/73/8e/5d/ecb50b90adaab5868ae1d8df180f31e55e85c2f055aaf2fb35
Successfully built fasttext
  Found existing installation: fasttext 0.9.1
    Uninstalling fasttext-0.9.1:
      Successfully uninstalled fasttext-0.9.1


In [0]:
overlap=30
def pred_correction_rnn(sent,model,dic,maxlen,wdim):
    conv = np.zeros((1,maxlen,wdim,1))
    rnn  = np.zeros((1,maxlen,wdim))
    charcount = -1
    for j in range(len(sent)):
      if j<maxlen and sent[j]!=' ':
        charcount=charcount+1
        conv[0][charcount,:,0]=dic[sent[j]]
        rnn[0][charcount,:]=dic[sent[j]]
    z = model.predict([conv,rnn])[0]
    sent_raw = ''
    count_char=-1
    lastpoint=-1
    lastchar=-1
    for j in range(len(sent)):
      if sent[j]!=' ':
        count_char=count_char+1
        sent_raw = sent_raw+sent[j]
        if z[count_char]>threshold_kor:
          sent_raw = sent_raw+' '
          if j<overlap:
            lastpoint=len(sent_raw)
            lastchar=j
    return sent_raw, lastpoint, lastchar
    
def kor_spacing(s):
    if len(s)<overlap:
      temp,lp,lc = pred_correction_rnn(s,model_kor,dic_kor,100,100)
      z = temp+"\n"
    else:
      z=''
      start=0
      while start<len(s):
        if start+overlap<len(s):
          temp,lp,lc =pred_correction_rnn(s[start:start+2*overlap],model_kor,dic_kor,100,100)
          temp=temp[:lp]
        else:
          temp,lp,lc =pred_correction_rnn(s[start:],model_kor,dic_kor,100,100)
          lc = overlap
        z = z+temp
        start=start+lc+1
      z = z+"\n"
    return z